## Analysis of stock prices using PCA / Notebook 3

In this notebook we will study the dimensionality of stock price sequences, and show that they lie between the 1D of smooth functions and 2D of rapidly varying functions.

The mathematicians Manuel Mandelbrot and Richard Hudson wrote a book titled [The Misbehavior of Markets: A Fractal View of Financial Turbulence](https://www.amazon.com/gp/product/0465043577?ie=UTF8&tag=trivisonno-20&linkCode=as2&camp=1789&creative=9325&creativeASIN=0465043577).  In this book they demonstrate that financial sequences have a fractal dimension that is higher than one. In other words, the changes in stock prices are more similar to random walk, than to a smooth differentiable curve.

In this notebook we will estimate the fractal dimension of sequences corresponding to the log of the price of a stock. We will do the same for some other, non-random sequences.

We will use the [Box Counting](https://en.wikipedia.org/wiki/Box_counting) method to estimate the dimension.

### Box Counting
For the sake of simplicity, lets start with a simple smooth curve corresoinding to $sin(x)$.
Intuitively speaking, the dimension of this curve should be 1. Lets see how we measure that using box-counting.

The idea is simple: we split the 2D plane into smaller and smaller rectangles and count the number of rectangles that touch the curve.  The gridlines in the figure below partition the figure into $16 \times 16 = 256$ rectangles. The yellow shading corresponds the partition of the figure into $8 \times 8$ rectangles. The green corresponds to the partition into $16\times 16$ (which is the same as the grid), The blue and the red correspond to partitions into $32\times32$ and $64 \times 64$ respectively. You can see that as theboxes get smaller their number increases. 

![Sinusoid](figs/Sinusoid.BoxCount.png)

The dimension is defined by the relation between the size of the cubes and the number of rectangle that touch the curve. More precisly, we say that the size of a rectangle in a $n \times n$ partition is $\epsilon=1/n$. We denote by $N(\epsilon)$ the number of rectangles of size $\epsilon$ that touch the curve. Then if $d$ is the dimension, the relationship between $N(\epsilon)$ and $\epsilon$ is 
$$
N(\epsilon) = \frac{C}{\epsilon^d}
$$
For some constant $C$

Taking $\log$s of both side we get 
$$
(1)\;\;\;\;\;\;\;\;\;\;\;\;\log N(\epsilon) = \log C + d \log \frac{1}{\epsilon}
$$

We can use this equation to estimate $d$ as follows: let $\epsilon_2 \gg \epsilon_1$ be two sizes that are far apart (say $\epsilon_1=1/4$ and $\epsilon_2=1/1024$), and let $N(\epsilon_1),N(\epsilon_2)$ be the corresponding box counts. Then by taking the difference between Equation (1) for the two sizes we get the estimate
$$
    d \approx \frac{\log N(\epsilon_1) - \log N(\epsilon_2)}{\log \epsilon_2- \log \epsilon_1}
$$

Note that this is an estimate, it depends on the particular values of $\epsilon_1$ and $\epsilon_2$. We can refer to it as the "dimension" if we get the same number for any choice of the two sizes (as well as other details sich as the extent of the function.

Here are similar figures for the seque 
![AMZN](figs/AMZN.BoxCount.png)

![IBM](figs/IBM.BoxCount.png)		

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

#sc.stop()
sc = SparkContext(master="local[3]")

from pyspark.sql import *
sqlContext = SQLContext(sc)

%pylab inline

import numpy as np


Populating the interactive namespace from numpy and matplotlib


In [2]:
df=sqlContext.read.csv('SP500.csv',header='true',inferSchema='true')
df.count()

13422

In [3]:
columns=df.columns
col=[c for c in columns if '_P' in c]
tickers=[a[:-2] for a in col]
tickers[:10],len(tickers)

(['AGN', 'EOG', 'WLP', 'CPB', 'YUM', 'JWN', 'FLS', 'TAP', 'BWA', 'AAPL'], 476)

In [4]:
def get_seq(ticker):
    key=ticker+"_P"
    L=df.select(key).collect()
    L=[x[key] for x in L if not x[key] is None]
    return L

#### You can generate graphs like the ones below for your analysis of dimensionality on the stocks (recommended but not graded)
![Graph for Analysing Stocks](figs/plots.png)

In [5]:
pickleFile="Tester/Dimensionality.pkl"

##  Exercise 1 -- Finding Dimension


In this exercise, you will do the following:
    * You will find the dimension for a particular ticker using its sequence of data

######  <span style="color:blue">Sample Input:</span>
```python
  
dimension = Box_count([sequence of AAPL], 'AAPL')

```
######  <span style="color:magenta">Sample Output:</span>
dimension = 1.28

In [6]:
# Method to find out the dimensionality of the sequence of a ticker
def getNe(LL, ticker, n):
    
    totalDays = len(LL)
    logPrice =  map(lambda x: log(x), LL)
    maxLogPrice = max(logPrice)
    minLogPrice = min(logPrice)
    xgrid = np.linspace(0,totalDays,n+1).tolist()
    ygrid = np.linspace(minLogPrice,maxLogPrice,n+1).tolist()
    
    xspace = xgrid[1]-xgrid[0]
    yspace = ygrid[1]-ygrid[0]
    
    countMatrix = np.zeros((n,n))
    
    boxpoints =  n+1
    
    for i in range(totalDays-1):
        x1 = i
        x2 = i+1
        y1 = logPrice[i]
        y2 = logPrice[i+1]
        xPoints = np.linspace(x1,x2,boxpoints).tolist()
        yPoints = np.linspace(y1,y2,boxpoints).tolist()
        for j in range(boxpoints):
            xindex = int(xPoints[j]/xspace)
            yindex = int((yPoints[j] - minLogPrice)/yspace) -1
            if(countMatrix[xindex][yindex] == 0):
                countMatrix[xindex][yindex] = 1
            
    return np.sum(countMatrix)
    


def Box_count(LL,ticker):
    ## Your Implementation goes here

    n1 = 4
    n2 = 512
    Ne1 = getNe(LL, ticker, n1)
    Ne2 = getNe(LL, ticker, n2)
    epsilon1 = 1.0/n1
    epsilon2 = 1.0/n2
    
    dimension = (log(Ne1) - log(Ne2))/(log(epsilon2) - log(epsilon1))    
    
    return dimension

### Test Cases for Finding Dimension

In [7]:
import Tester.Dimensionality as Dimensionality
Dimensionality.exercise_1(pickleFile, Box_count)

Input: sequence 1
Correct Output: 0.988354081412
Great Job!



In [8]:
import Tester.Dimensionality as Dimensionality
Dimensionality.exercise_2(pickleFile, Box_count)

Input: sequence 2
Correct Output: 1.98760591825
Great Job!



In [9]:
import Tester.Dimensionality as Dimensionality
Dimensionality.exercise_3(pickleFile, Box_count)

Input: sequence 3
Correct Output: 1.34156643681
Great Job!

